In [1]:
from IPython.core.display import HTML

center_string = """
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
"""

# Risk Budgeting

**Adapted from Econ 424/CFRM 462 - Portfolio Risk Budgeting - Eric Zivot - August 14, 2014**

Risk Budgeting additively decompose a measure of portfolio risk into contributions from the individual assets in the portfolio.
by show which assets are most responsible for portfolio risk in terms of volatility

In [2]:
import os
import sys

os.getcwd()

sys.path.insert(0, r'C:\Users\lucab\PycharmProjects\Warehouse')

In [3]:
from primitive import *
import matplotlib.pyplot as plt

In [4]:
ris = rebase_at_x(pd.read_excel('risk_budgeting.xlsx', sheet_name='ris', index_col='ref_date'))
weights_and_names = pd.read_excel('risk_budgeting.xlsx', sheet_name='weights', index_col='instrument_id')
weights = weights_and_names.weight
weights = pd.Series(np.array([0.1, 0.2, 0.3, 0.4]), weights.index)

FileNotFoundError: [Errno 2] No such file or directory: 'risk_budgeting.xlsx'

In [ ]:
fig = plt.figure(figsize=(9, 7))
ax = fig.add_subplot(111)
ris.plot(ax=ax, linewidth=1.5, legend=True)
ax.set_title('Time Series', fontsize=15)
plt.show()

HTML(center_string)

Return of a Portfolio in a certain time $t$ is simply defined as the weighted sum of each of its $n$ components return, i.e.

$$
\begin{equation}
\text{R}_p^{(t)} = \sum_{i}^{n} \text{w}_i^{(t)} \text{r}_i^{(t)}\tag{1}
\end{equation}
$$

In a two assets world:

$$
\text{R}_p^{(t)} = \text{w}_1^{(t)} \text{r}_1^{(t)} + \text{w}_2^{(t)} \text{r}_2^{(t)}
$$

Over time, portfolio return will be a vector of returns where each day's return is computed as in $(1)$:

$$
\text{R}_p = \text{w}_1 \text{r}_1 + \text{w}_2 \text{r}_2
$$


Its variance and standard deviation are respectivelyis (dropping the time notation):

$$
\sigma_p^2= \text{w}_1^{2} \sigma_1^{2} + \text{w}_2^{2} \sigma_2^{2} + 2\text{w}_1\text{w}_2\sigma_{12}
$$

$$
\sigma_p= \sqrt{\text{w}_1^{2} \sigma_1^{2} + \text{w}_2^{2} \sigma_2^{2} + 2\text{w}_1\text{w}_2\sigma_{12}}
$$



In [ ]:
rets = ris.pct_change().dropna()
rets = rets.loc[:, weights.index.tolist()]
std_devs = rets.std()
ptf_ret = rets.dot(weights)  # compute ptf returns
ptf_vol = ptf_ret.std()

In [ ]:
ptf_vol

In [ ]:
rets.cov()

In [ ]:
def get_MCR(vCov, w, ptf_std):
    MCR = w.dot(vCov) / ptf_std
    return MCR

def get_CR(MCR, w):
    CR = MCR * w
    return CR

def get_PCR(CR, ptf_std):
    PCR = CR / ptf_std
    return PCR

def risk_decomposition(vCov, w, ptf_std):
    MCR = get_MCR(vCov, w, ptf_std)
    CR = get_CR(MCR, w)
    PCR = CR / ptf_std
    return MCR, CR, PCR

def optimize_with_risk_budget_spinu(risk_budgets, omega):
    # Solve the problem where correlations are not taken into consideration,
    # i.e. omega is an array of standard deviations
    if len(omega.shape) == 1:
        y = np.sqrt(risk_budgets) / omega
        return y / sum(y)

In [5]:
w = weights.values
st_dev_only = False
if st_dev_only:
    PCR = risk_decomposition(np.diag(np.diag(rets.cov())), w, ptf_vol)[2]
else:
    PCR = risk_decomposition(rets.cov(), w, ptf_vol)[2]

NameError: name 'weights' is not defined

In [92]:
pd.Series(PCR * 100)

0     0.396595
1     4.798230
2     6.068320
3    88.736855
dtype: float64

In [97]:
optimize_with_risk_budget_spinu(PCR, rets.std())

15442     0.244508
28335     0.323626
29037     0.199895
165879    0.231971
dtype: float64

In [42]:
np.diag(rets.cov())

array([1.42934782e-06, 9.87119332e-06, 3.27219119e-05, 3.55315133e-04])

In [47]:
np.diag(np.diag(rets.cov()))

array([[1.42934782e-06, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 9.87119332e-06, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 3.27219119e-05, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.55315133e-04]])

In [95]:
rets.std()

15442     0.001196
28335     0.003142
29037     0.005720
165879    0.018850
dtype: float64